In [ ]:
# ==========================================
# Traffic Forecasting with Interactive User Input
# ==========================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# -----------------------
# 1. Simulate Traffic Data
# -----------------------
np.random.seed(42)
time_steps = 2000
traffic_data = np.sin(np.linspace(0, 50, time_steps)) + np.random.normal(0, 0.3, time_steps)
traffic_data = traffic_data.reshape(-1, 1)

# -----------------------
# 2. Create Sliding Windows
# -----------------------
IN_STEPS = 24   # use last 24 timesteps
OUT_STEPS = 6   # predict next 6 timesteps

def create_windows(series, in_steps, out_steps):
    X, y = [], []
    for i in range(len(series) - in_steps - out_steps):
        X.append(series[i:i+in_steps])
        y.append(series[i+in_steps:i+in_steps+out_steps])
    return np.array(X), np.array(y)

X, y = create_windows(traffic_data, IN_STEPS, OUT_STEPS)

# -----------------------
# 3. Train/Test Split
# -----------------------
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test   = X[train_size:], y[train_size:]

# -----------------------
# 4. Scaling
# -----------------------
scaler = MinMaxScaler()
X_train_flat = X_train.reshape(-1, 1)
X_test_flat  = X_test.reshape(-1, 1)

scaler.fit(X_train_flat)

X_train = scaler.transform(X_train_flat).reshape(X_train.shape)
X_test  = scaler.transform(X_test_flat).reshape(X_test.shape)

y_train = scaler.transform(y_train.reshape(-1,1)).reshape(y_train.shape)
y_test  = scaler.transform(y_test.reshape(-1,1)).reshape(y_test.shape)

# -----------------------
# 5. Build Model
# -----------------------
model = Sequential([
    LSTM(64, input_shape=(IN_STEPS, 1), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(OUT_STEPS)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train.squeeze(-1), epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# # -----------------------
# # 6. Prediction Function
# # -----------------------
# def predict_future(user_sequence):
#     if len(user_sequence) != IN_STEPS:
#         raise ValueError(f"Need exactly {IN_STEPS} values, got {len(user_sequence)}")

#     # scale input
#     user_seq_scaled = scaler.transform(np.array(user_sequence).reshape(-1,1)).reshape(1, IN_STEPS, 1)

#     # predict
#     pred_scaled = model.predict(user_seq_scaled)
#     pred = scaler.inverse_transform(pred_scaled.reshape(-1,1)).flatten()

#     return pred

# # -----------------------
# # 7. Interactive User Input
# # -----------------------
# # print(f"\nEnter {IN_STEPS} recent traffic values separated by commas.")
# # print("Or just press Enter to use dataset values.")

# # user_input = input("Your values: ")

# # if user_input.strip() == "":
# #     # fallback: use last 24 from dataset
# #     example_input = traffic_data[-IN_STEPS:].flatten().tolist()
# #     print("\nUsing last 24 values from dataset...")
# # else:
# #     try:
# #         example_input = [float(x.strip()) for x in user_input.split(",")]
# #     except:
# #         raise ValueError("Invalid input. Please enter numeric values separated by commas.")

# # future_pred = predict_future(example_input)

# # print("\nPredicted Next 6 Timesteps:")
# # print(future_pred)

# # Plot
# plt.figure(figsize=(8,4))
# plt.plot(range(IN_STEPS), example_input, label="Input (last 24)")
# plt.plot(range(IN_STEPS, IN_STEPS+OUT_STEPS), future_pred, 'r--', label="Predicted (next 6)")
# plt.legend()
# plt.title("Traffic Forecast from User Input")
# plt.show()


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1899 - val_loss: 0.0199
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0192 - val_loss: 0.0143
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0152 - val_loss: 0.0112
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0142 - val_loss: 0.0114
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0132 - val_loss: 0.0113
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0132 - val_loss: 0.0116
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0128 - val_loss: 0.0114
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0118 - val_loss: 0.0107
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0119 - val_loss: 0.0112
Epoch 10/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0116 - val_loss: 0.0109


In [ ]:
# Example routes with predicted congestion
routes = {
    "Route A": [0.6, 0.7, 0.8],  # higher traffic
    "Route B": [0.2, 0.3, 0.4],  # lighter
    "Route C": [0.5, 0.4, 0.6]   # moderate
}

# Compute average congestion
avg_congestion = {r: sum(vals)/len(vals) for r, vals in routes.items()}

# Best route = minimum congestion
best_route = min(avg_congestion, key=avg_congestion.get)

print("Best Route:", best_route)

# Suggest public transport if all routes are congested
if avg_congestion[best_route] > 0.7:
    print("Suggestion: Use Metro/Bus instead of Car 🚍")
else:
    print("Car is fine 🚗")

Best Route: Route B
Car is fine 🚗


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pickle

# Save model
with open("citypulse_gbr_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save scaler
with open("citypulse_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ Model and Scaler saved successfully!")

✅ Model and Scaler saved successfully!
